In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 19.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=34c6f117726b15c203c8b0f233bf05a717afb9e3c9c6a15a6602119672b6b878
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
import pyspark
import pandas as pd
from pyspark import SparkConf, SparkContext
from nltk.util import ngrams
from random import shuffle, randint
import math, time

In [33]:
sc = SparkContext(master="local" , appName="LSH")
#sc.stop()

In [126]:
def hashString(text):
  return abs(hash(text)) % (10 ** 8)

In [127]:
data = sc.textFile ("dataset.tsv")
header = data.first() #extract header
#remove header
data = data.filter(lambda row : row != header) 
data = data.map(lambda line: line.split("\t"))
#just get title and description
data = data.map(lambda itemInLine: itemInLine[0]+itemInLine[1])
data.collect()[0]

"Studio accessoriato vicino metro A Furio Camillo Affitto studio a professionisti preferibilmente medico sanitari, Metro A fermata Furio Camillo 100 metri. Fermata bus 87 e 625 10 metri, fronte Villa Lazzaroni. Luminoso, finemente ristrutturato, pavimenti in parquet con aria condizionata autonoma doppio uso, luci a led, connessione internet wi-fi. E' previsto uso bagno, sala d'attesa, area ristoro, tutto con pavimento in parquet. E' possibile condividere lo studio con altri professionisti se gradito. Villa Lazzaroni ampia possibilitÃ\xa0 di parcheggio. "

Shingles

In [128]:
def splitToken(word, kGram):
  specialWords = '[\/.@]'
  text = word.lower()
  #remove special word
  for word in specialWords:
    text = text.replace(word, ' ')
  nGram = getData(ngrams(text.split(), kGram))
  shinglesText = []
  for grams in nGram:
    shinglesText.append(' '.join(grams))
  return shinglesText

def getData(gen):
      try:
        for elem in gen:
          yield elem
      except (RuntimeError, StopIteration):
          return

In [130]:
shinglingSize = 10
#all shingles
listShingles = data.flatMap(lambda line : splitToken(line, shinglingSize))
listShingles = listShingles.map(lambda eachShingles : ''.join(eachShingles))
listShingles = listShingles.distinct()

#shingles of each docs
listShingesOfEachDocs = data.map(lambda line : splitToken(line, shinglingSize))

In [131]:
print("Tranfer singles")
startTime = time.time()
shingles = listShingles.collect()
shingesOfEachDocs = listShingesOfEachDocs.collect()
endTime = time.time()
print("Time: "+str((endTime-startTime)))
#print(listShingles.count())

Tranfer singles
Time: 3.037964344024658


In [132]:
listShingles.collect()[1]

'accessoriato vicino metro a furio camillo affitto studio a professionisti'

Min Hashing

In [133]:
class MinHash:
  def __init__(self, sig_size, listShingles):
    self.sig_size = sig_size
    self.listShingles = listShingles
    self.hash_functions = []
    self.createPermuation()
    #print(self.hash_functions)
  
  #create array permutation
  def createPermuation(self):
    #create array from 1 to len(listShingles)
    for i in range(0,self.sig_size):
      arrayNumberInPermution = list(range(1,len(self.listShingles)+1))
      shuffle(arrayNumberInPermution)
      self.hash_functions.append(arrayNumberInPermution)
  
  def createMaxtrixTrueFalse(self, s):
    data = {}
    columnName = 'set' + str(1)
    arrayTrueFalse = []
    print(s)
    for shingle in self.listShingles:
      if (shingle in s):
        arrayTrueFalse.append(1)
      else:
        arrayTrueFalse.append(0)
    data[columnName] = arrayTrueFalse
    #print(pd.DataFrame(data))
    return pd.DataFrame(data) 

  def computeSetSignature(self, singlesOfDoc):
    set_sig = []
    maxtrixTrueFalse = self.createMaxtrixTrueFalse(singlesOfDoc)
    #just get only row is 1
    maxtrixTrue = maxtrixTrueFalse[maxtrixTrueFalse['set1']==1]
    #run each column in permutation and check get value min
    for columnPermutation in self.hash_functions:
      min_hash = math.inf
      for index, row in maxtrixTrue.iterrows():
          if (columnPermutation[index] < min_hash):
            min_hash = columnPermutation[index]
      set_sig.append(min_hash)
    return set_sig

In [135]:
signatureSize = 10
minHash = MinHash(signatureSize, listShingles.collect())
singatureMaxtrix = listShingesOfEachDocs.map(lambda setShingle: minHash.computeSetSignature(setShingle))

In [136]:
print("Tranfer Min Hash")
startTime = time.time()
signature = singatureMaxtrix.collect()
endTime = time.time()
print("Time: "+str((endTime-startTime)))

Tranfer Min Hash
Time: 469.226548910141


In [137]:
signature[0]

[475, 767, 960, 1571, 1064, 180, 780, 1530, 702, 5126]

LSH

In [138]:
class LSH:
  def __init__(self, threshold=0.8):
    self.threshold = threshold
  
  #create band with number of bands (bands_nr)
  #divide item in maxtrix into bands_nr of bands
  def divideMatrixToNBands(self, sigMatrix, bandsNr, signLen):
    r = int(signLen/bandsNr)#number of rows in each band
    bands = {}
    for i in range(0, bandsNr):
      bands[i] = []
    #print(sigMatrix[:10])
    for signature in sigMatrix:
      for i in range(0, bandsNr):
        idx = i*r
        for x in signature[idx:idx+r]:
          bands[i].append(str(x))
    #print(bands)
    return bands
  
  #hash item in band into buckets
  def hashItemToBucketOfBand(self, band):
    buckets = {}
    #docId is index of doc in dataset
    for docId in range(0,len(band)):
      value = hashString(band[docId])
      if value not in buckets:
        buckets[value] = [docId]
      else:
        buckets[value].append(docId)
    #print(buckets)
    return buckets

  #pair group
  def groupPairItemCandidate(self, buckets):
    candidates = set()
    #print("-----------------------------------------------")
    #print(buckets)
    for bucket, candidateList in buckets.items():
      if (len(candidateList) > 1):
        for i in range(0, len(candidateList)-1):
          for j in range(i+1, len(candidateList)):
            pair = tuple(sorted((candidateList[i], candidateList[j])))
            candidates.add(pair)
    #print("Candidates")
    #print(candidates)
    return candidates #ie a set of couples, each couple is a candidate pair
  
  #compair docs
  def checkCandidates(self, candidatesList, threshold, sigs):
    similar_docs = set()

    for similarPair in candidatesList:
      docId1 = similarPair[0]
      docId2 = similarPair[1]
      #print(doc_id_1)
      signature1 = set(sigs[docId1])
      signature2 = set(sigs[docId2])
      js = len(signature1.intersection(signature2)) / len(signature1.union(signature2))

      if js >= threshold:
        similar_docs.add(tuple(sorted((docId1, docId2))))
    return similar_docs
  
  def getSimilarItems(self, sigMatrix, bandsNr, signLen):
    similarDocs = set()
    bands = self.divideMatrixToNBands(sigMatrix, bandsNr, signLen)
    #print(bands)
    for band_id, elements in bands.items():
      buckets = self.hashItemToBucketOfBand(elements)
      candidates = self.groupPairItemCandidate(buckets)
      for sim_tuple in self.checkCandidates(candidates, self.threshold, sigMatrix):
        similarDocs.add(sim_tuple)
        
    return similarDocs

In [139]:
print("Tranfer LSH")
startTime = time.time()
bandsNr = 10
lsh = LSH(threshold =0.8)
lshSimilarItem = lsh.getSimilarItems(signature, bandsNr, signatureSize)
endTime = time.time()
print("Time: "+str((endTime-startTime)))

Tranfer LSH
Time: 0.7497096061706543


In [140]:
for i in range(0,3):
  docs = lshSimilarItem.pop()
  print("Doc ",docs[0]," :")
  print(data.collect()[docs[0]])
  print("Doc ",docs[1]," :")
  print(data.collect()[docs[1]])
  print("------------------------------------------------------------------------------------")

Doc  506  :
Avviato studio psicologia mette a disposizione stanze Avviato studio di psicologia mette a disposizione stanze a professionisti della salute (psicologi, medici, nutrizionisti, counselor, fisioterapisti, ecc). Centralissimo (adiacente Metro Spagna), finemente arredato, silenzioso e luminoso. Pulizie e utenze incluse. Si valuta inoltre possibilitÃ  di sinergia professionale 
Doc  568  :
Avviato studio psicologia mette a disposizione stanze Avviato studio di psicologia mette a disposizione stanze a professionisti della salute (psicologi, medici, nutrizionisti, counselor, fisioterapisti, ecc). Centralissimo (adiacente Metro Spagna), finemente arredato, silenzioso e luminoso. Pulizie e utenze incluse. Si valuta inoltre possibilitÃ  di sinergia professionale 
------------------------------------------------------------------------------------
Doc  89  :
Negozio 169Mq per laboratorio, ufficio, studio a Roma. Privato affitta negozio 169 mq, al piano terra, ristrutturato, 4 serrande